In [8]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import os
%cd /cluster2/home/futing/Project/HiCQTL/merged/CRC53/covariate
prefix='CRC53hard'

/cluster2/home/futing/Project/HiCQTL/merged/CRC53/covariate


In [23]:
order=pd.read_csv('/cluster2/home/futing/Project/HiCQTL/merged/CRC53/genotype/CRC53hard/list',sep=' ',header=None)

#### 合并 genoPC 和 cooler info

In [ ]:

genoPC=pd.read_csv(f'{prefix}geno.eigenvec',sep='\t') #,header=None

# genoPC=genoPC.drop(columns=[0]) # 去掉第一列
genoPC.columns=['cell']+["PC" + str(i) for i in range(1,11)] # 设置列名
genoPC.head()

,cell,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
0,09-376,-0.003505,0.045586,-0.004631,-0.020398,-0.020335,0.032894,-0.021555,-0.002999,-0.001834,-0.038123
1,11-1026,-0.042613,0.009025,-0.016630,-0.013799,-0.008520,0.024014,-0.028614,0.010684,-0.006534,-0.024911
2,11-1074,-0.034777,0.052751,-0.084831,-0.070476,-0.134842,0.258324,-0.237561,-0.515049,-0.067692,0.147545
3,11-1074_Normal,-0.010388,0.072458,-0.087327,-0.074125,-0.138394,0.261997,-0.240741,-0.516256,-0.066955,0.140087
4,11-51,-0.015184,0.070601,-0.013332,-0.170600,-0.227671,0.270144,0.446353,0.157856,-0.242540,0.082860


In [14]:
coolinfo=pd.read_csv('/cluster2/home/futing/Project/panCancer/QC/nContacts/hicInfo_Aug04.txt',sep='\t')
coolinfo['element']=coolinfo['element'].str.replace(',','')

In [16]:
covariate=pd.merge(coolinfo[['cell','element']],genoPC,on='cell',how='right')
covariate=covariate.set_index('cell')
covariate['tumor'] = [0 if x.endswith('_Normal') else 1 for x in covariate.index]
# covariate = covariate.T
covariate.head(8)

,element,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,tumor
cell,,,,,,,,,,,,
09-376,105179165,-0.003505,0.045586,-0.004631,-0.020398,-0.020335,0.032894,-0.021555,-0.002999,-0.001834,-0.038123,1
11-1026,102757999,-0.042613,0.009025,-0.016630,-0.013799,-0.008520,0.024014,-0.028614,0.010684,-0.006534,-0.024911,1
11-1074,95497276,-0.034777,0.052751,-0.084831,-0.070476,-0.134842,0.258324,-0.237561,-0.515049,-0.067692,0.147545,1
11-1074_Normal,90275817,-0.010388,0.072458,-0.087327,-0.074125,-0.138394,0.261997,-0.240741,-0.516256,-0.066955,0.140087,0
11-51,61174254,-0.015184,0.070601,-0.013332,-0.170600,-0.227671,0.270144,0.446353,0.157856,-0.242540,0.082860,1
11-51_Normal,113095102,0.006134,0.086450,-0.015566,-0.174204,-0.233101,0.274247,0.457002,0.162277,-0.248086,0.084484,0
11-52,94723091,0.032578,0.097371,0.036266,-0.336717,0.577875,-0.036173,0.158797,-0.116986,0.062458,0.003769,1
11-52_Normal,87713161,0.004171,0.086851,0.031656,-0.317500,0.526104,-0.025250,0.139781,-0.107140,0.058179,-0.000222,0


### 合并 phenoPC genoPC 和 cooler info

In [ ]:

filelist=pd.read_csv(f'../genotype/{prefix}/{prefix}.psam',sep='\t') #genotype的顺序
filelist['#IID'].equals(order.T[0])


True

In [65]:
expr=pd.read_csv('/cluster2/home/futing/Project/HiCQTL/merged/CRC53/phenotype/CRC53_Aug15.bed',sep='\t')
fixed = expr.iloc[:, :4]
reordered = expr.loc[:, filelist['#IID']]
expr_new = pd.concat([fixed, reordered], axis=1)

expr_new.head()

,#chr,start,end,gene_id,09-376,11-1026,11-1074,11-1074_Normal,11-51,11-51_Normal,...,14-91,15-1382,15-1420,15-418,15-446,16-178,16-178_Normal,DKO,HCT116,SW480
0,chr1,890000,895000,178,0.721850,0.901089,0.839639,0.330144,0.312873,0.727936,...,0.256476,1.384033,0.912626,0.379787,0.659389,0.668441,1.320379,0.998001,1.662335,0.989169
1,chr1,895000,900000,179,0.540577,0.595470,0.650683,0.389669,-0.254923,0.668293,...,0.210331,0.905053,0.605062,0.557428,0.368272,0.516904,0.907322,0.891386,1.520709,0.562468
2,chr1,900000,905000,180,0.418510,0.642061,0.561610,0.451869,-0.214719,0.916896,...,-0.009144,0.866584,0.759631,0.540593,0.420482,0.781214,0.543994,0.787745,1.590608,0.251199
3,chr1,905000,910000,181,0.497176,0.356007,0.261403,0.214934,0.068536,0.341432,...,-0.015539,0.695639,0.733940,0.361112,0.640313,0.426678,-0.188751,0.682265,1.415113,0.359788
4,chr1,910000,915000,182,0.814550,0.834990,0.563191,0.274900,0.074303,0.271685,...,0.370163,0.901901,0.828751,0.348402,0.670816,0.333805,-0.137275,0.779230,1.365818,0.674101


In [69]:
expr_new.to_csv('/cluster2/home/futing/Project/HiCQTL/merged/CRC53/phenotype/CRC53_Aug15_index.bed',sep='\t')

In [30]:
df=expr.drop(columns=['#chr','start','end','gene_id']).T
# df=expr.drop(columns=['gene_id']).T
# 1. 标准化数据（PCA 对量纲很敏感）
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)

# 2. 建立 PCA 模型，保留前 2 个主成分
pca = PCA(n_components=20)
X_pca = pca.fit_transform(X_scaled)

In [31]:
# 整理列名
expr_pc = pd.DataFrame(X_pca)
expr_pc.columns = ["exprPC" + str(i) for i in range(1,21)]
expr_pc.index = df.index
# expr_pc = expr_pc.T
expr_pc.head()

,exprPC1,exprPC2,exprPC3,exprPC4,exprPC5,exprPC6,exprPC7,exprPC8,exprPC9,exprPC10,exprPC11,exprPC12,exprPC13,exprPC14,exprPC15,exprPC16,exprPC17,exprPC18,exprPC19,exprPC20
09-376,-32.255252,5.779546,60.482489,26.834561,40.021673,-53.615280,105.084984,-66.093430,-168.249042,-74.899292,3.105812,67.639939,62.034776,-116.538205,94.239799,215.523808,303.216050,-205.460556,407.549029,7.623603
11-1026,20.911479,2.904043,-13.723535,-38.171587,-37.298422,-37.995354,30.237122,-41.071728,28.702975,-27.843897,30.262943,-54.275845,-72.711726,-2.395463,-24.473173,-8.254037,45.464840,-210.357637,-90.773052,86.361572
11-1074,27.369037,-14.722040,-32.431884,-41.141936,-10.995128,-48.376539,11.072289,12.933012,43.691512,22.317427,6.796151,67.075384,-106.329067,33.436372,34.238993,46.871371,30.786566,-101.945079,38.939496,-69.289557
11-1074_Normal,-227.068299,-34.736590,16.147323,92.318391,16.885811,17.009664,4.805006,-45.347539,-36.881989,-13.234147,-59.944864,-19.498039,9.531484,-30.285611,-48.255263,-13.694456,-90.484372,75.198537,-26.926289,1.306994
11-51,13.187841,1013.483443,-233.743933,83.144315,46.770182,51.488661,-22.890348,2.872613,9.102910,4.110900,-7.636565,6.564034,8.951921,9.161821,-9.323533,-2.694236,-9.545628,13.425760,0.755142,-0.124317


In [32]:
covariate1=pd.merge(covariate,expr_pc,    left_index=True, # 使用左边 DataFrame 的 index 作为合并键
    right_index=True,# 使用右边 DataFrame 的 index 作为合并键
    how='inner')

In [72]:
covariate2=covariate1.reindex(filelist['#IID'])
covariate2.index.equals(pd.Index(filelist['#IID']))
covariate2.index.name = 'ID'

In [77]:
# covariate1=covariate1.T
covariate2.T.to_csv('/cluster2/home/futing/Project/HiCQTL/merged/CRC53/covariate/covariate27.txt',sep='\t',index=True)

In [78]:
covariate2.head()

,element,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,...,exprPC11,exprPC12,exprPC13,exprPC14,exprPC15,exprPC16,exprPC17,exprPC18,exprPC19,exprPC20
ID,,,,,,,,,,,,,,,,,,,,,
09-376,105179165,-0.003505,0.045586,-0.004631,-0.020398,-0.020335,0.032894,-0.021555,-0.002999,-0.001834,...,3.105812,67.639939,62.034776,-116.538205,94.239799,215.523808,303.216050,-205.460556,407.549029,7.623603
11-1026,102757999,-0.042613,0.009025,-0.016630,-0.013799,-0.008520,0.024014,-0.028614,0.010684,-0.006534,...,30.262943,-54.275845,-72.711726,-2.395463,-24.473173,-8.254037,45.464840,-210.357637,-90.773052,86.361572
11-1074,95497276,-0.034777,0.052751,-0.084831,-0.070476,-0.134842,0.258324,-0.237561,-0.515049,-0.067692,...,6.796151,67.075384,-106.329067,33.436372,34.238993,46.871371,30.786566,-101.945079,38.939496,-69.289557
11-1074_Normal,90275817,-0.010388,0.072458,-0.087327,-0.074125,-0.138394,0.261997,-0.240741,-0.516256,-0.066955,...,-59.944864,-19.498039,9.531484,-30.285611,-48.255263,-13.694456,-90.484372,75.198537,-26.926289,1.306994
11-51,61174254,-0.015184,0.070601,-0.013332,-0.170600,-0.227671,0.270144,0.446353,0.157856,-0.242540,...,-7.636565,6.564034,8.951921,9.161821,-9.323533,-2.694236,-9.545628,13.425760,0.755142,-0.124317
